In [ ]:
%pip install scikit-learn

In [ ]:

"""
TF‑IDF Retrieval-Augmented Generation Pipeline 
This script:
  1. Loads CSV files (application, employee, organisation, process, etc.)
  2. Creates document strings for each entity.
  3. Builds a TF‑IDF representation of all documents.
  4. Reads queries from an Excel file.
  5. For each query and for each specified top-k value:
       • Retrieves the top‑k documents (using TF‑IDF cosine similarity)
       • Generates a final answer using an LLM 
       • Records retrieval and LLM generation (response) times.
  6. Writes the results (retrieved document texts, times, final answer) into an output Excel file.

"""

import os
import time
import json
import argparse
import pandas as pd
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from types import SimpleNamespace 
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from openai import AzureOpenAI 

# -------------------------------
# 1. Load CSV Files and Create Documents
# -------------------------------
app_file = r"data/application_master.csv"   # Application data
app_emp_file = r"data/apps_owners.csv"       # Application connecting with employees
emp_file = r"data/Fin_Emp.csv"               # Employee data
org_file = r"data/Fin_Org.csv"               # Organisation connecting with employees
proc_file = r"data/process_master.csv"     # Process data
proc_org_file = r"data/orgs_processes.csv" # Process connecting with organisation
proc_app_file = r"data/process_applications.csv"  # Processes connecting applications
proc_emp_file = r"data/process_owners.csv"       # Process connecting with employees

print("Loading CSV files...")
app_df      = pd.read_csv(app_file)
app_emp_df  = pd.read_csv(app_emp_file)
emp_df      = pd.read_csv(emp_file)
org_df      = pd.read_csv(org_file)
proc_df     = pd.read_csv(proc_file)
proc_org_df = pd.read_csv(proc_org_file)
proc_app_df = pd.read_csv(proc_app_file)
proc_emp_df = pd.read_csv(proc_emp_file)

In [2]:

def create_document(entity_type, row, extra_info=""):
    """Flatten a pandas Series into a document string."""
    fields = [f"{col}: {row[col]}" for col in row.index if pd.notnull(row[col])]
    doc_text = f"{entity_type.upper()} DATA: " + " | ".join(fields)
    if extra_info:
        doc_text += " | " + extra_info
    metadata = {"entity_type": entity_type, "id": row.get("id", None)}
    return {"doc_id": f"{entity_type}_{row.get('id', '')}", "text": doc_text, "metadata": metadata}

documents = []


In [3]:

# --- Application Documents ---
for _, row in app_df.iterrows():
    extra_parts = []
    linked_emp = app_emp_df[app_emp_df['app_id'] == row['id']]
    if not linked_emp.empty:
        emp_info = []
        for _, lrow in linked_emp.iterrows():
            emp_info.append(f"employee_id: {lrow['employee_id']} (is_owner: {lrow['is_owners']})")
        extra_parts.append("Linked employees: " + ", ".join(emp_info))
    extra_parts.append("App Org: " + str(row.get("app_org", "")))
    extra = " | ".join(extra_parts)
    documents.append(create_document("application", row, extra))

# --- Employee Documents ---
for _, row in emp_df.iterrows():
    extra = f"Org ID: {row.get('org_id', '')}, Line Manager ID: {row.get('line_manager_id', '')}"
    documents.append(create_document("employee", row, extra))

# --- Organisation Documents ---
for _, row in org_df.iterrows():
    extra = f"Org Head: {row.get('org_head', '')}, Parent Org ID: {row.get('parent_org_id', '')}"
    documents.append(create_document("organisation", row, extra))

# --- Process Documents ---
for _, row in proc_df.iterrows():
    extra_parts = []
    linked_org = proc_org_df[proc_org_df['process_id'] == row['id']]
    if not linked_org.empty:
        org_ids = [f"org_id: {r['org_id']}" for _, r in linked_org.iterrows()]
        extra_parts.append("Linked Organisation(s): " + ", ".join(org_ids))
    linked_app = proc_app_df[proc_app_df['process_id'] == row['id']]
    if not linked_app.empty:
        app_ids = [f"application_id: {r['application_id']}" for _, r in linked_app.iterrows()]
        extra_parts.append("Linked Application(s): " + ", ".join(app_ids))
    linked_emp = proc_emp_df[proc_emp_df['process_id'] == row['id']]
    if not linked_emp.empty:
        emp_ids = [f"employee_id: {r['employee_id']} (is_owner: {r['is_owners']})" for _, r in linked_emp.iterrows()]
        extra_parts.append("Linked Employee(s): " + ", ".join(emp_ids))
    extra = " | ".join(extra_parts)
    documents.append(create_document("process", row, extra))

print(f"Created {len(documents)} documents.")

# Create a list of document texts
doc_texts = [doc["text"] for doc in documents]


Created 195537 documents.


In [ ]:

# -------------------------------
# 2. Build TF‑IDF Retrieval Pipeline
# -------------------------------

print("Building TF‑IDF index...")
tfidf_vectorizer = TfidfVectorizer(max_features=15000, stop_words="english")
doc_tfidf = tfidf_vectorizer.fit_transform(doc_texts) 
print("TF‑IDF index built.")

def retrieve_tfidf(query, top_k=5):
    """Retrieve top-k documents for a query using cosine similarity on TF‑IDF vectors."""
    query_vec = tfidf_vectorizer.transform([query])
    # Calculate cosine similarity
    cosine_sim = (doc_tfidf * query_vec.T).toarray().flatten()
    top_indices = np.argsort(cosine_sim)[::-1][:top_k]
    results = [documents[i] for i in top_indices]
    return results


In [ ]:

# --------------------------------------------------- 
# 3. LLM Functions (Azure OpenAI) 
# --------------------------------------------------- 
def load_config(): 
    config_path = r"config.json" 
    try: 
        with open(config_path) as f: 
            config = json.load(f, object_hook=lambda d: SimpleNamespace(**d)) 
        return config 
    except FileNotFoundError: 
        raise FileNotFoundError("Config file not found. Please check the path.") 

def initialize_azure_client(config): 
    """Initialize Azure KeyVault and Azure OpenAI client.""" 
    client = SecretClient(vault_url=config.key_vault_url, credential=DefaultAzureCredential()) 
    secret = client.get_secret(config.dev_secret_name) 
    return AzureOpenAI(api_key=secret.value, 
                    api_version=config.chat.api_version, 
                    azure_endpoint=config.chat.azure_endpoint) 


def generate_answer_with_llm(query: str, top_documents): 
    """
    Use Azure OpenAI to generate a final answer from the top retrieved documents. 
    """ 
    config = load_config() 
    llm = initialize_azure_client(config) 
    context = "\n\n".join(top_documents) 
    prompt = [ 
        { 
            "role": "system", 
            "content": f""" 
You are an AI assistant tasked with answering a query based on the provided context about employees and organizations. 
Please provide a detailed and well-structured answer to the user's question. 

- Organize the answer into bullet points if appropriate. 
- Use headings where relevant. 
- Include all relevant details concisely. 

Context: 
{context} 

Question: "{query}" 

Provide a well-structured answer. 
            """ 
        } 
    ] 
    response = llm.chat.completions.create(model=config.chat.model, messages=prompt) 
    response_content = response.choices[0].message.content.strip() 
    return response_content 


In [ ]:
# Example test query
example_query = "What is the email address of Anubhuti Singh?"

# Retrieve top-k documents for the example query
retrieved_docs = retrieve_tfidf(example_query, top_k=5)

print("Retrieved Documents :")
for doc in retrieved_docs:
    print(f"Doc ID: {doc['doc_id']}")
    print(doc["text"][:150] + "...")
    print("-" * 50)

# Generate an answer using the LLM simulation function
top_docs_texts = [doc["text"] for doc in retrieved_docs]
generated_answer = generate_answer_with_llm(example_query, top_docs_texts)

print("\nGenerated Answer:")
print(generated_answer)


In [ ]:

# -------------------------------
# 4. Process Queries from Excel File
# -------------------------------

query_excel_file = r"data/LLMEval_1.xlsx"      
output_excel_file = r"Outputs/LLM_responses_tf-idf_multihop.xlsx"

print("Loading query Excel file...")
queries_df = pd.read_excel(query_excel_file)

# List of k values to test.
k_values = [1, 3, 5, 8, 13, 15, 21]

# For each query, for each k, retrieve docs, generate final answer, and record timings.
for idx, row in queries_df.iterrows():
    query = row["Query"]
    for k in k_values:
        col_docs = f"k_{k}_docs"
        col_retrieval_time = f"k_{k}_retrieve_time"
        col_response_time = f"k_{k}_response_time"
        col_final_answer = f"k_{k}_final_answer"
        # Retrieve documents using TF‑IDF.
        start_time = time.time()
        retrieved_docs = retrieve_tfidf(query, top_k=k)
        retrieval_time = time.time() - start_time
        # Prepare a string with the top retrieved document texts.
        docs_str = "\n\n----\n\n".join([doc["text"] for doc in retrieved_docs])
        top_docs_texts = [doc["text"] for doc in retrieved_docs]
        # Generate answer using the LLM.
        start_time = time.time()
        final_answer = generate_answer_with_llm(query, top_docs_texts)
        print(final_answer)
        response_time = time.time() - start_time
        # Save results in the DataFrame.
        queries_df.at[idx, col_docs] = docs_str
        queries_df.at[idx, col_retrieval_time] = retrieval_time
        queries_df.at[idx, col_response_time] = response_time
        queries_df.at[idx, col_final_answer] = final_answer
        print(f"Processed query '{query[:50]}...' for k={k}: retrieval {retrieval_time:.2f}s, response {response_time:.2f}s.")

# -------------------------------
# 5. Save Results to Excel
# -------------------------------
queries_df.to_excel(output_excel_file, index=False)
print(f"Results saved to {output_excel_file}")
